<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# Colab launcher — Ensures UI actually starts and prints LOCAL + PUBLIC URLs
# Paste entire cell and run in Colab
import os, subprocess, shutil, re, time
from pathlib import Path
from google.colab import drive

# ---------- CONFIG ----------
REPO_ZIP_URL = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")
SYMLINK_MAP = {
    "models": "models",
    "loras": "loras",
    "training": "training",
    "user_data/characters": "characters",
    "user_data/presets": "presets",
    "user_data/prompts": "prompts",
    "user_data/settings.yaml": "settings/settings.yaml",
    "user_data/settings.json": "settings/settings.json",
    "user_data/chat": "chat-history",
    "user_data/instruct": "instruct-history",
    "outputs": "outputs",
    "user_data/images": "images",
    "user_data/logs": "logs",
    "user_data/cache": "cache",
    "user_data/extensions": "extensions",
    "user_data/softprompts": "softprompts",
}
# launch commands to try (first is preferred)
PREFERRED_LAUNCH = "bash start_linux.sh --share --listen --api"
FALLBACK_LAUNCH = "python server.py --share --listen --auto-launch"
# ----------------------------

def run(cmd, cwd=None, env=None, check=False):
    return subprocess.run(cmd, shell=True, cwd=cwd, env=env, text=True,
                          capture_output=True, check=check)

def safe_remove(path: Path):
    try:
        if path.is_symlink() or path.is_file():
            path.unlink(missing_ok=True)
        elif path.is_dir():
            shutil.rmtree(path, ignore_errors=True)
    except Exception:
        pass

def create_symlink(local: Path, target: Path):
    try:
        if local.exists() or local.is_symlink():
            safe_remove(local)
        local.parent.mkdir(parents=True, exist_ok=True)
        target.parent.mkdir(parents=True, exist_ok=True)
        # If linking a file, touch target
        if target.suffix and not target.exists():
            target.touch(exist_ok=True)
        local.symlink_to(target.resolve())
        print(f"✓ Linked: {local} → {target}")
    except Exception as e:
        print(f"⚠ Failed to link {local} -> {target}: {e}")

# 1) Mount Drive
print("▶ Mounting Google Drive")
drive.mount("/content/drive", force_remount=False)

# 2) Ensure Drive root and folders
print("▶ Ensuring Drive persistence folders")
DRIVE_ROOT.mkdir(parents=True, exist_ok=True)
for v in SYMLINK_MAP.values():
    p = DRIVE_ROOT / v
    if p.suffix:
        p.parent.mkdir(parents=True, exist_ok=True)
        p.touch(exist_ok=True)
    else:
        p.mkdir(parents=True, exist_ok=True)

# 3) Download/extract repo if missing
if not WORK_DIR.exists():
    print("▶ Downloading repository (zip fallback)")
    try:
        run("rm -f /content/repo.zip")
        r = run(f"wget -O /content/repo.zip {REPO_ZIP_URL}")
        if r.returncode != 0 or not Path("/content/repo.zip").exists():
            # try git clone as alternative
            print("  wget failed; trying git clone fallback")
            r2 = run(f"git clone --depth 1 https://github.com/gitleon8301/MY-AI-Gizmo-working.git {WORK_DIR}")
            if r2.returncode != 0:
                raise RuntimeError("Failed to fetch repository by zip and git")
        else:
            run("unzip -q /content/repo.zip -d /content")
            extracted = next(Path("/content").glob("MY-AI-Gizmo-working-*"), None)
            if not extracted:
                raise RuntimeError("Extraction succeeded but expected folder missing")
            extracted.rename(WORK_DIR)
        print("✓ Repository obtained")
    except Exception as e:
        print("ERROR: cannot fetch repository:", e)
        raise SystemExit("Repository required — aborting")
else:
    print("✓ Repository already present")

os.chdir(WORK_DIR)

# 4) Create symlinks into repo for persistence
print("▶ Creating symlinks for persistence")
for local_rel, drive_name in SYMLINK_MAP.items():
    local_path = WORK_DIR / local_rel
    drive_target = DRIVE_ROOT / drive_name
    # Ensure drive target exists
    if drive_target.suffix:
        drive_target.parent.mkdir(parents=True, exist_ok=True)
        drive_target.touch(exist_ok=True)
    else:
        drive_target.mkdir(parents=True, exist_ok=True)
    create_symlink(local_path, drive_target)

# 5) Environment fixes & detect GPU
print("▶ Applying environment fixes")
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
os.environ['MPLBACKEND'] = 'Agg'
if 'PYTHONPATH' in os.environ:
    del os.environ['PYTHONPATH']

has_gpu = False
try:
    import torch
    has_gpu = torch.cuda.is_available()
    print("▶ GPU available:", has_gpu)
except Exception:
    print("▶ torch not available; proceeding (CPU)")

env = os.environ.copy()
env['GPU_CHOICE'] = 'A' if has_gpu else 'C'
env['LAUNCH_AFTER_INSTALL'] = 'FALSE'   # ensure installer doesn't auto-launch
env['INSTALL_EXTENSIONS'] = 'FALSE'
env['MPLBACKEND'] = 'Agg'

# 6) Run installer once (non-fatal)
print("▶ Running installer (start_linux.sh). This may print 'Will now exit due to LAUNCH_AFTER_INSTALL.' — that's expected.")
if (WORK_DIR / "start_linux.sh").exists():
    (WORK_DIR / "start_linux.sh").chmod((WORK_DIR / "start_linux.sh").stat().st_mode | 0o111)
    installer_proc = run("bash start_linux.sh", cwd=WORK_DIR, env=env)
    print(installer_proc.stdout or "")
    if installer_proc.returncode == 0:
        print("✓ Installer finished (exit code 0)")
    else:
        print("⚠ Installer returned non-zero (continuing; launch will be attempted separately)")
else:
    print("⚠ start_linux.sh not found — installer step skipped")

# 7) Pin Gradio to known-compatible version (required)
print("\n🔧 Pinning Gradio to compatible version")
run("pip uninstall -y gradio gradio_client", check=False)
pin_proc = run("pip install -q gradio==3.41.2 gradio_client==0.5.0", check=False)
print(pin_proc.stdout or pin_proc.stderr or "✓ pip executed (see output above).")
print("✓ Gradio pin attempt complete — continue to launch\n")

# 8) Launch server (try preferred, fallback to python server.py)
def stream_process_and_extract(proc, timeout_seconds=600):
    """Stream proc stdout; extract local/public URL; return tuple(local, public, exitcode)."""
    start = time.time()
    local_url = None
    public_url = None
    installer_exit_message_seen = False
    local_patterns = [re.compile(r"Running on local URL:\s*(http://[^\s]+)"),
                      re.compile(r"(http://(?:127\.0\.0\.1|localhost|0\.0\.0\.0):\d+/?\S*)")]
    public_patterns = [re.compile(r"(https://[a-z0-9\-]+\.gradio\.live\S*)"),
                       re.compile(r"(https://[a-z0-9\-]+\.gradio\.app\S*)"),
                       re.compile(r"(https://[a-z0-9\-]+\.trycloudflare\.com\S*)"),
                       re.compile(r"(https://[a-z0-9\-]+\.ngrok\.io\S*)")]
    try:
        while True:
            line = proc.stdout.readline()
            if line == "" and proc.poll() is not None:
                break
            if not line:
                if time.time() - start > timeout_seconds:
                    break
                time.sleep(0.1)
                continue
            print(line, end="")

            # Detect installer-exit message which indicates start_linux.sh didn't launch UI
            if "Will now exit due to LAUNCH_AFTER_INSTALL" in line:
                installer_exit_message_seen = True

            if not local_url:
                for p in local_patterns:
                    m = p.search(line)
                    if m:
                        local_url = m.group(1)
                        break
            if not public_url:
                for p in public_patterns:
                    m = p.search(line)
                    if m:
                        public_url = m.group(1)
                        break
                # fallback heuristic
                if not public_url:
                    m2 = re.search(r"(https://[^\s]+)", line)
                    if m2 and "gradio" in m2.group(1):
                        public_url = m2.group(1)

            # If both found, return immediately (but keep print)
            if local_url or public_url:
                print("\n" + "="*72)
                if local_url:
                    print(f"📍 LOCAL  : {local_url}")
                if public_url:
                    print(f"🌍 PUBLIC : {public_url}")
                print(f"💾 SAVED  : {DRIVE_ROOT}")
                print("="*72 + "\n")
                return local_url, public_url, None, installer_exit_message_seen

    except KeyboardInterrupt:
        print("\n▶ KeyboardInterrupt received; terminating process")
        try:
            proc.terminate()
        except Exception:
            proc.kill()

    return local_url, public_url, proc.poll(), installer_exit_message_seen

# Kill stray servers
run("pkill -f 'python server.py' || true")
run("pkill -f 'gradio' || true")
time.sleep(1)

# Try preferred launch
print("▶ Attempting preferred launch:", PREFERRED_LAUNCH)
proc = subprocess.Popen(PREFERRED_LAUNCH, shell=True, cwd=WORK_DIR, env=env,
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
local, public, exitcode, saw_installer_exit = stream_process_and_extract(proc, timeout_seconds=240)

if (local or public) and exitcode is None:
    print("▶ UI launched successfully using preferred command.")
else:
    # If preferred command exited early or printed installer-exit, try fallback
    print("▶ Preferred launch did not produce a UI (exitcode={}, installer-exit={}). Trying fallback.".format(exitcode, saw_installer_exit))
    try:
        # ensure any previous proc is terminated
        try:
            proc.terminate()
        except Exception:
            pass
        time.sleep(1)
    except Exception:
        pass

    if Path(WORK_DIR / "server.py").exists():
        print("▶ Attempting fallback launch:", FALLBACK_LAUNCH)
        proc2 = subprocess.Popen(FALLBACK_LAUNCH, shell=True, cwd=WORK_DIR, env=env,
                                 stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
        local2, public2, exitcode2, _ = stream_process_and_extract(proc2, timeout_seconds=240)
        if (local2 or public2) and exitcode2 is None:
            print("▶ UI launched successfully using fallback command.")
            local, public = local2, public2
        else:
            print("⚠ Fallback also failed or timed out. Inspect output above for errors.")
    else:
        print("⚠ No server.py found for fallback. Inspect output above for errors and ensure the repo supports non-docker launch.")

# Final status
if not local and not public:
    print("\n⚠ UI URLs not found. The logs above contain the first error to address.")
else:
    print("\n✅ Done — use the PUBLIC or LOCAL URL printed above to open the UI.")

print(f"\n▶ Persistent Drive folder: {DRIVE_ROOT}")
